<a href="https://colab.research.google.com/github/Murcha1990/ML_AI24/blob/main/Hometasks/Base/AI_HW6_uplift.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1> Задание по Uplift-моделированию </h1>

<h2>Введение</h2>

Перед вами типичная задача, возникающая при работе с моделями кампейнинга в банке: заказчик запустил несколько пилотов по взаимодействию с клиентами с помощью разных каналов: push в мобильном приложении, sms, баннеры в мобильном приложении и реклама в других приложениях экосистемы. Заказчик хотел бы понимать, какой канал взаимодействия с клиентом наиболее эффективен для каждого клиента из клиентской базы. Кампании планируются и запускаются в ежемесячном режиме. Иными словами, заказчик хотел бы в идеале ежемесячно получать список клиентов, которым необходимо отправить коммуникацию с указанием канала и прироста вероятности покупки в случае, если клиенту отправят коммуникацию по сравнению с тем случаем, когда клиенту коммуникацию не отправят.

<b>Таким образом: </b>
1.	У нас есть база клиентов (клиенты, имеющие id в банке). По данной базе осуществляется рассылка тех или иных стимулирующих коммуникаций по различным продуктам, каналам (например SMS, Push, баннеры в мобильном приложении и т.д.) и сегментам клиентов
2.	Признаковое описание клиента состоит из различных агрегатов действий клиента за месяц или его объективных характеристик: например, средняя сумма средств на депозитах за месяц, среднее число кликов клиента в день за месяц в разделе "инвестиции" в мобильном приложении или возраст клиента
3.	При формировании обучающей/тестовой выборки допускается, что один и тот же клиент за разные месяцы — это разные объекты. То есть допускается, что клиент в феврале и клиент в марте — это разные клиенты (то есть мы можем оперировать с ними как с разными сущностями).
4.	Агрегаты действий клиента за месяц появляются примерно 10 числа следующего месяца. То есть, например, агрегаты за декабрь появляются 10 января. В свою очередь списки клиентов, которым необходимо осуществить рассылку должны быть сформированы ориентировочно 20 числа предыдущего месяца. Таким образом, <b> модель должна быть обучена делать предсказания с лагом в два месяца </b>, то есть должна делать предсказание на март по клиентским агрегатам за январь. Обязательно учтите это при обучении модели (в противном случае можно получить лик таргета, так как часто величину, которую мы предсказываем уже есть в клиентских агрегатах, но смещенная на два месяца).


## Оценивание задания:

Всего за задание можно получить 50 первичных баллов, которые затем переводятся в 10-балльную шкалу делением не 5.

Скачаем архив с данными по ссылке и разархивируем.

In [1]:
# !pip install gdown -q

In [2]:
# import gdown

# url = 'https://drive.google.com/uc?id=19nKGaxm3RwHxh2UWPo537_-MDx21AkHO'
# output = 'Data.zip'
# gdown.download(url, output, quiet=False)

In [3]:
# import zipfile

# with zipfile.ZipFile(output, 'r') as zip_ref:
#     zip_ref.extractall('./')

<h2>Описание данных</h2>

Перед вами несколько наборов данных, на основе которых вам будет необходимо обучить Uplift модели, сделать прогноз на нужный месяц и решить, кому из клиентов отправлять коммуникацию, а кому коммуникацию отправлять не следует.

<h3>Features </h3> Признаки клиентов, клиентские агрегаты, которые описывают поведение клиентов <br>

1. user_id - id клиента
2. report_dt - месяц, на который актуальны признаки
3. city - город, в котором живет клиент
4. age - возраст клиента
5. x1 – x9 - числовые признаки клиента, характеризующие поведение клиента

Первичный ключ таблицы - user_id + report_dt

<h3> Contracts </h3> Таблица с покупками продуктов.

1. contract_id - id покупки
2. user_id - id пользователя, который совершил покупку
3. product_id - id продукта, который был куплен
4. contract_ts – дата момента, когда была совершена покупка

Первичный ключ - contract_id


<h3> Campaings </h3> Кампании, которые проводились (под кампанией мы понимаем рассылку sms, push и т.д).

1. campaing_id - id кампании, первичный ключ таблицы
2. product_id - продукт, по которому проводилась кампания (считаем, что продукты не конкурируют друг с другом)
3. channel - канал, в котором проводилась кампания


<h3> People_in_campaings </h3> Люди, которые принимали участие в кампаниях.

1. campaing_id - id кампании
2. user_id - id пользователя, который попал в кампанию
3. флаг целевой (1) и контрольной (0) группы (целевая группа - это те, кто получил коммуникацию, а контрольная - те, кто нет)
4. delivery_ts - timestamp, когда клиенту фактически была доставлена коммуникация (для контрольной группы nan, подумайте почему)

Первичный ключ данной таблицы - user_id + campaing_id


<h3> Contracts </h3> Таблица с покупками продуктов

1. contract_id - id покупки
2. user_id - id пользователя, который совершил покупку
3. product_id - id продукта, который был куплен
4. contract_ts – дата момента, когда была совершена покупка

Первичный ключ - contract_id


<h1> Постановка задачи </h1> В ноябре 2024 проводилось несколько кампаний по продукту с id 0001 (фактически клиенту рассылалось одно и тоже сообщение, но в разных каналах). Вам необходимо по данным кампаниям построить модель, которая будет определять лучший канал коммуникации каждого клиента и определить, кому из клиентов в марте 2025 отправить какую коммуникацию, а кому коммуникацию вообще отправлять не следует.
Ответ нужно представить в следующем виде (report_dt – дата фичей):

<table>
  <thead>
    <tr>
      <th>user_id</th>
      <th>report_dt</th>
      <th>channel</th>
      <th>uplift</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>10045</td>
      <td>2025-01-31</td>
      <td>banner</td>
      <td>0.07</td>
    </tr>
    <tr>
      <td>10046</td>
      <td>2025-01-31</td>
      <td>no_comm</td>
      <td>0.00</td>
    </tr>
    <tr>
      <td>10047</td>
      <td>2025-01-31</td>
      <td>sms</td>
      <td>0.23</td>
    </tr>
    <tr>
      <td>10048</td>
      <td>2025-01-31</td>
      <td>push</td>
      <td>0.19</td>
    </tr>
  </tbody>
</table>

<h1> Декомпозиция задачи </h1>

In [1]:
import pandas as pd

<h2> 1.	Сбор и анализ таргета (18 баллов)</h2>

Прежде всего, вам необходимо собрать целевое событие, которое вы собираетесь прогнозировать. В данном случае целевое событие - это покупка продукта 0001 пользователем, участвовавшем в кампании. Обратите внимание, что не все пользователи получают коммуникацию одновременно (delivery_ts в таблице People_in_campaings). Согласно правилу, согласованному с заказчиком, <b> человек из целевой группы купил продукт после коммуникации - это значит, что он купил его в течение 2х недель после получения сообщения, а человек из контрольной - в течение 3х недель с момента старта кампании (старт кампании - начало месяца). </b> То есть для определенной кампании, для каждого клиента, попавшего в кампанию, вам надо будет найти его покупки данного продукта, а потом основываяся на данном правиле превратить покупки в 0 или 1. <br> На выходе у вас должен появиться таблица с целевым действием для каждого канала (колонки client_id, report_dt,  target), где таргет - это бинарная переменная (0 или 1). Колонка report_dt вам нужна как техническая колонка для дальнейших джоинов.<br><br>

Проведите анализ полученных данных (до присоединения клиентских агрегатов). Какие проблемы и сложности в данных вы обнаружили? Что с ними можно сделать? Какая из кампаний наиболее эффективная? Подготовьте выводы по полученным инсайтам.


**Комментарий по заданиям и оцениванию:**

* Вы должны самостоятельно сделать join нескольких таблиц, самостоятельно собрать целевое действие

* Представлены 4 различных канала, за таргет по каждому из каналов можно получить **максимум 2 балла**:
    * 1 балл за то, что просчитано целевое действие для целевой группы (покупка в
течение одной-двух недель с момента получения коммуникации)
    * 1 балл за то, что просчитано целевое действие для контрольной группы (покупка в течение двух-трех недель с момента старта кампании) и сделана таблица в требуемом формате

* Обратите внимание, что не во всех кампаниях содержатся корректные данные для проведения моделирования, и вам необходимо провести анализ данных и в случае выявленных некорректностей - описать их, и не проводить моделирование для "сломанной" кампании  
    * За данный анализ можно получить **8 баллов**

* Вы должны оценить эффективность кампаний по uplift (cреднее значение таргета в целевой минус среднее значение таргета в контрольной группе)
    * За данный анализ можно получить **2 балла**

In [2]:
features = pd.read_csv('data/AGGS_FINAL.csv').drop('Unnamed: 0', axis=1)
contracts = pd.read_csv('data/CONTRACTS_FINAL.csv').drop('Unnamed: 0', axis=1)
campaigns = pd.read_csv('data/CAMPAINGS.csv').drop('Unnamed: 0', axis=1)
people_in_campaigns = pd.read_csv('data/PEOPLE_IN_CAMPAINGS_FINAL.csv').drop('Unnamed: 0', axis=1)

: 

In [ ]:
features.head()

In [ ]:
features.info()

`Нужно привести report_dt в формат даты.`

In [ ]:
features['report_dt'] = pd.to_datetime(features['report_dt'], format='%Y-%m-%d')

In [ ]:
features['report_dt'].describe()

`В этом атрибуте информативен только месяц.`

In [ ]:
features['user_id'].unique().shape[0] == features.shape[0]

`user_id принимает неуникальные значения.`

In [ ]:
contracts.head()

In [ ]:
contracts['product_id'].unique()

`product_id принимает только одно значения, cоответсвующие продукту 0001. Этот атрибут можно удалить.`

In [ ]:
contracts.drop('product_id', axis=1, inplace=True)

In [ ]:
contracts.info()

`Нужно привести contract_date в формат даты.`

In [ ]:
contracts['contract_date'] = pd.to_datetime(contracts['contract_date'], format='%Y-%m-%d')

In [ ]:
contracts['contract_date'].describe()

`Все покупки осуществлялись в ноября.`

In [ ]:
contracts['user_id'].unique().shape[0] == contracts.shape[0]

`Клиенты совершали только одну покупку в течение ноября.`

In [ ]:
campaigns.head()

In [ ]:
campaigns['product_id'].unique()

`product_id принимает только одно значение, соответсвующие продукту 0001. Этот атрибут можно удалить.`

In [ ]:
campaigns.drop('product_id', axis=1, inplace=True)

In [ ]:
people_in_campaigns.head()

`delivery_date нужно привести к типу даты, значения unknown принять за пропуск.`

In [ ]:
people_in_campaigns['delivery_date'] = pd.to_datetime(people_in_campaigns['delivery_date'], format='%Y-%m-%d', errors='coerce')

In [ ]:
people_in_campaigns['delivery_date'].describe()

`Компании проводились только в ноябре.`

`Объединим people_in_campaigns и campaigns.`

In [ ]:
people_in_campaigns_merge_campaigns = people_in_campaigns.merge(right=campaigns,
                          how='left',
                          on='campaing_id',
                          validate='many_to_one')

people_in_campaigns_merge_campaigns

`Отделим контрольную группу от целевой.`

In [ ]:
target_campaigns = people_in_campaigns_merge_campaigns[~people_in_campaigns_merge_campaigns['delivery_date'].isna()]
control_campaigns = people_in_campaigns_merge_campaigns[people_in_campaigns_merge_campaigns['delivery_date'].isna()].drop('delivery_date', axis=1)

In [ ]:
target_campaigns

`Проверка по полю t_flag.`

In [ ]:
target_campaigns['t_flag'].unique()

In [ ]:
control_campaigns

`Проверка по полю t_flag.`

In [ ]:
control_campaigns['t_flag'].unique()

In [ ]:
len(set(control_campaigns['user_id'].unique()).intersection(set(target_campaigns['user_id'].unique())))

`60000 клиентов попали в обе группы одновременно.`

In [ ]:
for campaign in campaigns['campaing_id']:
    print(f'{campaign}:')
    filter_target = (target_campaigns['campaing_id'] == campaign)
    filter_control = (control_campaigns['campaing_id'] == campaign)
    intersection_len = len(set(control_campaigns[filter_control]['user_id']).intersection(set(target_campaigns[filter_target]['user_id'])))
    print(f'intersection length: {intersection_len}')

`В рамках одной компании пересечений нет, следовательно нужно искать пересечения между разными компаниями.`

In [ ]:
people_in_campaigns_merge_campaigns_pivot = people_in_campaigns_merge_campaigns.copy()

people_in_campaigns_merge_campaigns_pivot['campaing_id_'] = people_in_campaigns_merge_campaigns_pivot['campaing_id']

pd.pivot_table(data=people_in_campaigns_merge_campaigns_pivot,
               index='campaing_id',
               columns='campaing_id_',
               values='user_id',
               aggfunc=lambda x: len(x.unique()))

### ваши выводы здесь

<h2> 2. Клиентские агрегаты (12 баллов)</h2>

Присоедините клиентские агрегаты (будьте внимательны, присоедините агрегаты за корректный месяц) и изучите полученные данные.

**Комментарий по заданиям и оцениванию:**

* Вы должны корректно присоединить клиентские агрегаты со смещением на два месяца, чтобы не было лика таргета. За данное действие можно получить **4 балла**

* Далее вы должен сделать UPLIFT EDA, которые обсуждались на лекции и показывались в практических ноутбуках. В ходе анализа вы должны проверить корректность данных по рекламным кампаниям и решить, что делать со "сломанными" кампаниями. По итогам анализа подготовьте выводы. За данное действие можно получить **8 баллов**

`Получим признаки для клиентов в ноябре.`

In [ ]:
features_november = features[(features['report_dt'].dt.year == 2024) & (features['report_dt'].dt.month == 11)]

In [ ]:
features_november['user_id'].unique().shape[0] == features_november.shape[0]

`Клиенты в ноябре уникальны.`

In [ ]:
# ваш код здесь

### ваши выводы здесь

<h2> 3. Построение моделей и оценка их качества (14 баллов)</h2>

Постройте Uplift модели по собранным кампаниям, проведите тюнинг гиперпараметров и оцените их качество (qini score). Для каждой модели также постройте qini-curve.

**Комментарий по заданиям и оцениванию:**

* Реализован только подход Solomodel без дополнительных библиотек и калибровок  - **1 балл**

* Реализован Solomodel или Twomodel через Sklift или CausalML - **2 балла**

* Учтена калибровка Metalearner'ах - **2 балла**

* Корректно реализован ClassTransformation - **2 балла**

* Реализован UpliftRandomForest - **4 балла**

* Использованы пайплайны в Sklift - **2 балла**

* Реализован тюнинг ( Gridsearch \ Optuna ) - **1 балл**

In [ ]:
# ваш код здесь

<h2>4. Подготовка ответа в требуемом формате и подготовка выводов (6 баллов)</h2>

a) Сделайте скоринг нужных клиентов, подготовьте ответ в требуемом формате

б) Сделайте краткую аналитику того, какой канал взаимодействия наиболее предпочтителен

в) Сделайте выводы по проделанной работе

**Комментарий по заданиям и оцениванию:**

* Подготовлен только ответ - **1 балл**
* Подготовлен содержательный вывод по проделанной работе - **4 балла**
* Корректно принято решение об отправке/не отправке коммуникации клиентам в зависимости от значений Uplift - **1 балл**

In [ ]:
# ваш код здесь

### ваши выводы здесь